In [2]:
%matplotlib inline

In [3]:
import numpy as np
import pandas as pd

In [4]:
# check files in directory
from subprocess import check_output
print(check_output(["ls"]).decode("utf8"))

Kaggle-HomeDepot.ipynb
attributes.csv
product_descriptions.csv
relevance_instructions.docx
sample_submission.csv
test.csv
train.csv
zipfiles



In [5]:
# load files
training_data = pd.read_csv("train.csv")
testing_data = pd.read_csv("test.csv")
attribute_data = pd.read_csv("attributes.csv")
descriptions = pd.read_csv("product_descriptions.csv")


In [6]:
# look at the top 10 rows
print(training_data.tail(10))
print(attribute_data.head(10))
print(descriptions.head(10))

           id  product_uid                                      product_title  \
74057  221434       206621  Crown Bolt M6-32 x 90 mm. Internal Hex Socket ...   
74058  221443       206627           25 in. Stainless Tip-Out Sink Front Tray   
74059  221449       206631  Masonite New Haven Three Quarter Oval Lite Pri...   
74060  221450       206632  Lilly Miller UltraGreen 1 Gal. Vitamin B-1 Pla...   
74061  221455       206637  Schluter Rondec Stainless Steel 3/8 in. x 1 in...   
74062  221457       206638  Atlantic Windowpane 576 CD or 192 DVD Blu-Ray ...   
74063  221458       206639  Philips 40-Watt Halogen R20 Flood Light Bulb (...   
74064  221463       206641  Schlage Camelot In-Active Aged Bronze Handlese...   
74065  221471       206648  Plastec 11 in. x 24 in. Rose Garden Wall Decor...   
74066  221473       206650  LICHTENBERG Pool Blue No. 918 Millennial Ryan ...   

                                      search_term  relevance  
74057                               m6 screw 

In [7]:
# merge descriptions
training_data = pd.merge(training_data, descriptions, on="product_uid", how="left")

In [12]:
training_data.head(10)

,id,product_uid,product_title,search_term,relevance,product_description
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,"Not only do angles make joints stronger, they ..."
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,"Not only do angles make joints stronger, they ..."
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00,BEHR Premium Textured DECKOVER is an innovativ...
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33,Update your bathroom with the Delta Vero Singl...
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67,Update your bathroom with the Delta Vero Singl...
5,18,100006,Whirlpool 1.9 cu. ft. Over the Range Convectio...,convection otr,3.00,Achieving delicious results is almost effortle...
6,20,100006,Whirlpool 1.9 cu. ft. Over the Range Convectio...,microwave over stove,2.67,Achieving delicious results is almost effortle...
7,21,100006,Whirlpool 1.9 cu. ft. Over the Range Convectio...,microwaves,3.00,Achieving delicious results is almost effortle...
8,23,100007,Lithonia Lighting Quantum 2-Light Black LED Em...,emergency light,2.67,The Quantum Adjustable 2-Light LED Black Emerg...
9,27,100009,House of Fara 3/4 in. x 3 in. x 8 ft. MDF Flut...,mdf 3/4,3.00,Get the House of Fara 3/4 in. x 3 in. x 8 ft. ...


In [14]:
# merge product counts
product_counts = pd.DataFrame(pd.Series(training_data.groupby(["product_uid"]).size(), name="product_count"))
training_data = pd.merge(training_data, product_counts, left_on="product_uid", right_index=True, how="left")

In [21]:
# merge brand names
# ...and change column name by .rename()
brand_names = attribute_data[attribute_data.name == "MFG Brand Name"][["product_uid", "value"]].rename(columns={"value": "brand_name"})
training_data = pd.merge(training_data, brand_names, on="product_uid", how="left")

In [40]:
# Check if any column has NaN
print(training_data.isnull().sum())
# Which rows are NaN
rowsnan = np.where(training_data['brand_name'].isnull())[0]
training_data.ix[rowsnan].head(10)

id                         0
product_uid                0
product_title              0
search_term                0
relevance                  0
product_description        0
product_count              0
brand_name             17556
dtype: int64


,id,product_uid,product_title,search_term,relevance,product_description,product_count,brand_name
198,660,100109,Shark Navigator Lift-Away Deluxe Bagless Vacuu...,shark cleaner,3.00,A lightweight and steerable Never Loses Suctio...,2,NaN
199,661,100109,Shark Navigator Lift-Away Deluxe Bagless Vacuu...,shark vacuum,3.00,A lightweight and steerable Never Loses Suctio...,2,NaN
219,712,100124,"Champion Power Equipment 3,100 Watt Gasoline P...",inverter generator,3.00,The Champion Power Equipment 16590 gasoline po...,1,NaN
302,969,100160,DEWALT 20-Volt Max Lithium-Ion Cordless Combo ...,dewalt combo kit,3.00,The DEWALT DCK940D2 combo kit is ideal for jus...,1,NaN
881,2808,100490,Milwaukee Metal Hole Saw Kit (15-Piece),$ hole saw,3.00,The Milwaukee Hole Dozer Bi-Metal Hole Saw fea...,1,NaN
1208,3773,100660,EGO 20 in. 56-Volt Lithium-Ion Walk-Behind Ele...,ego batteries,2.00,The performance of gas without the noise mess ...,1,NaN
1219,3800,100666,Eclipse Tools Heat Shrink Tubing - Assorted Co...,heat shrink tubing,3.00,"Used for electrical insulation, bundling, and ...",1,NaN
1241,3878,100678,Trinity EcoStorage 36 in. x 14 in. 3-Tier Bamb...,bakers rack,3.00,Trinity's EcoStorage Bamboo Baker's Rack is a ...,2,NaN
1242,3882,100678,Trinity EcoStorage 36 in. x 14 in. 3-Tier Bamb...,warehouse racks with shelves,2.33,Trinity's EcoStorage Bamboo Baker's Rack is a ...,2,NaN
1419,4371,100750,"GE 4.6 cu. ft. Top Load Washer in White, ENERG...",ge washer,3.00,Your GE ENERGY STAR 4.6 cu. ft. DOE Top Load W...,1,NaN


In [41]:
# fill in NaN
training_data.brand_name.fillna("Unknown", inplace=True)

In [44]:
# structure of the dataframe
print(str(training_data.info()))
# basic statistics of each column
print(str(training_data.describe()))
training_data[:50]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74067 entries, 0 to 74066
Data columns (total 8 columns):
id                     74067 non-null int64
product_uid            74067 non-null int64
product_title          74067 non-null object
search_term            74067 non-null object
relevance              74067 non-null float64
product_description    74067 non-null object
product_count          74067 non-null int64
brand_name             74067 non-null object
dtypes: float64(1), int64(3), object(4)
memory usage: 5.1+ MB
None
                  id    product_uid     relevance  product_count
count   74067.000000   74067.000000  74067.000000   74067.000000
mean   112385.709223  142331.911553      2.381634       1.935450
std     64016.573650   30770.774864      0.533984       1.706594
min         2.000000  100001.000000      1.000000       1.000000
25%     57163.500000  115128.500000      2.000000       1.000000
50%    113228.000000  137334.000000      2.330000       1.000000
75%    16827

,id,product_uid,product_title,search_term,relevance,product_description,product_count,brand_name
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,"Not only do angles make joints stronger, they ...",2,Simpson Strong-Tie
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,"Not only do angles make joints stronger, they ...",2,Simpson Strong-Tie
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00,BEHR Premium Textured DECKOVER is an innovativ...,1,BEHR Premium Textured DeckOver
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33,Update your bathroom with the Delta Vero Singl...,2,Delta
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67,Update your bathroom with the Delta Vero Singl...,2,Delta
5,18,100006,Whirlpool 1.9 cu. ft. Over the Range Convectio...,convection otr,3.00,Achieving delicious results is almost effortle...,3,Whirlpool
6,20,100006,Whirlpool 1.9 cu. ft. Over the Range Convectio...,microwave over stove,2.67,Achieving delicious results is almost effortle...,3,Whirlpool
7,21,100006,Whirlpool 1.9 cu. ft. Over the Range Convectio...,microwaves,3.00,Achieving delicious results is almost effortle...,3,Whirlpool
8,23,100007,Lithonia Lighting Quantum 2-Light Black LED Em...,emergency light,2.67,The Quantum Adjustable 2-Light LED Black Emerg...,1,Lithonia Lighting
9,27,100009,House of Fara 3/4 in. x 3 in. x 8 ft. MDF Flut...,mdf 3/4,3.00,Get the House of Fara 3/4 in. x 3 in. x 8 ft. ...,1,House of Fara
